### Implementation of the codes developed, openTURNS style


In [1]:
import openturns as ot
import numpy as np

We will use again the example developed, concerning the bending beam with random structural parameters. 
There will be some key differences with the first pipeline, as we will limit ourselves to a subset of the cases treated, and  not write some 'omnipotent' code.

In this new case, we will study the influence of each compônent of an aggreagated process (a collection of processes  each defined on a same shaped mesh), on a other aggregated process, by computing sobol sobol indices. 

To mimic other behaviours we can : 
    - Define a scalar random variable as a constant field or vector
    
###### Class will be intialized as so:
myAnalysis = ot.newSaltelliSensitivityAlgorithm(AggregatedProcess, FieldFunction, N, computeSecondOrder) 

This beahviour is based on the existing class: ot.SaltelliSensitivityAlgorithm






ot.RandomGenerator.SetSeed(0) 

baseDim = 1 #dimension of our process, the beam is 1D


# we have 2 processes in our problem, initially
#For young modulus:
amplitude = [5000]
scale = [300]
nu = 13/3
Model = MaternModel 

#For diameter:


In [2]:
# First step : DEFINING THE PROCESSES AND RANDOM VARIABLES
# First Process E_ : Youngs modulus
# dimension 
dim = 1
#grid
t0 = 0.0
step = 1000/101
N = 101
grid = ot.RegularGrid(t0, step, N)
#Covariance model Young's modulus
amplitude0 = [50000]*dim
scale0 = [300]*dim
nu0 = 13/3
model0 = ot.MaternModel(scale0, amplitude0, nu0)
# Karhunen Loeve decomposition of process 
algorithm = ot.KarhunenLoeveP1Algorithm(grid, model0, 1e-3)
algorithm.run()
resultsE = algorithm.getResult()
resultsE.setName('E_')

# Second Process D_ : Diameter
amplitude = [.3]*dim
scale = [250]*dim
nu = 13/3
model1 = ot.MaternModel(scale, amplitude, nu)
algorithm = ot.KarhunenLoeveP1Algorithm(grid, model1, 1e-3)
algorithm.run()
resultsD = algorithm.getResult()
resultsD.setName('D_')

def variablesAsProcess(distribution, mesh):
    '''Function to transform a scalar distribution into 
    a constant process defined over a mesh
    '''
    basis = ot.Basis([ot.SymbolicFunction(['x'],['1'])])
    lawAsprocess = ot.FunctionalBasisProcess(distribution, basis, mesh)
    lawAsprocess.setName(distribution.getName())
    return lawAsprocess

# First we define the distributions
# random variable for the density of the material (kg/m³)
sigma       = 750
nameD       = 'Rho'
RV_Rho = ot.Normal(0, sigma)
RV_Rho.setName(nameD)
# random variable for the position of the force   (mm) 
sigma_f      = 50
namePos     = 'FP'
RV_Fpos = ot.Normal(0, sigma_f)
RV_Fpos.setName(namePos)
# random variable for the norm of the force    (N)
sigma_Fnor    = 5.5
nameNor       = 'FN'
RV_Fnorm  = ot.Normal(0, sigma_Fnor)
RV_Fnorm.setName(nameNor)

# Then convert the distributions t processes over a mesh
SP_Rho = variablesAsProcess(RV_Rho, grid)
SP_Fpos = variablesAsProcess(RV_Fpos, grid)
SP_Fnorm = variablesAsProcess(RV_Fnorm, grid)

# Then we can do the Karhunen Loeve decomposition of the distribution
algorithm0 = ot.KarhunenLoeveP1Algorithm(grid, SP_Rho.getCovarianceModel(), 1e-3)
algorithm0.setNbModes(1)
algorithm0.run()
resultsRho = algorithm0.getResult()
resultsRho.setName('Rho_')

algorithm1 = ot.KarhunenLoeveP1Algorithm(grid, SP_Fpos.getCovarianceModel(), 1e-3)
algorithm1.setNbModes(1)
algorithm1.run()
resultsFpos = algorithm1.getResult()
resultsFpos.setName('Fpos_')

algorithm2 = ot.KarhunenLoeveP1Algorithm(grid, SP_Fnorm.getCovarianceModel(), 1e-3)
algorithm2.setNbModes(1)
algorithm2.run()
resultsFnor = algorithm2.getResult()
resultsFnor.setName('Fnorm_')

# Finally we get a list of aggregated KarhunenLoeve results
listOfKLRes = [resultsE, resultsD, resultsRho, resultsFpos, resultsFnor]


### NOTE :

When defining a distribution as a process, as shown above, it is of utmost importance to modifiy his function so that it takes as an input a constant process, not a distribution anymore. This can also be easily avoided, if you take the first value of the field generated.

###### Always multiple ways to proceed :

In the example presented here, 2 ways of approaching the problem are possible. First we can create all our processes individually, decompose them individually with Karhunen Loeve, and pass a list of KarhunenLoeveResults to our new function wrapper. 
But this is only necessary if we would have meshes of different shapes, or if we would have defined the scalar distributions on smaller meshes. But as all processes are defined over the same mesh here, we could have aggregated the Processes, and directly decompose the AggregatedProcess. 

###### Note2 :

When using the **KarhunenLoeveP1Algorithm** function, only the covariance function is decomposed this means **all data 'bout the means and the trend functions are FORGOTTEN**. You have to add these constants **Directly in yout function!** 


In [3]:
## Here we are going to define the function that is going to transform the list of processes 
## into a list of processes and random variables: 
def addConstant2ProcessSample(PS, constant):
    msh = PS.getMesh()
    NewPS = ot.ProcessSample(msh, PS.getSize(), PS.getDimension())
    newSamples = [PS[i] + constant for i in range(PS.getSize()) ]
    [NewPS.setField(ot.Field(msh, newSamples[i]), i) for i in range(PS.getSize())]
    return NewPS
        
def convertInputs(inputList):
    if isinstance(inputList[0],ot.ProcessSample):
        field_E = inputList[0]
        field_D = inputList[1]
        var_Rho  = [inputList[0][i].getMax()[0] for i in range(inputList[0].getSize())]
        var_Fpos = [inputList[1][i].getMax()[0] for i in range(inputList[1].getSize())] 
        var_Fnor = [inputList[2][i].getMax()[0] for i in range(inputList[2].getSize())]
        return addConstant2ProcessSample(field_E,210000),  addConstant2ProcessSample(field_D,10), var_Rho+7850, var_Fpos+500,  var_Fnor+100        
    else :
        field_E = inputList[0].getValues()
        field_D = inputList[1].getValues()
        var_Rho = inputList[2].getValues().getMax()[0] if inputList[2].getValues().getMin()[0]-inputList[2].getValues().getMin()[0]<1e-4 else print('Problem')  
        var_Fpos = inputList[3].getValues().getMax()[0] if inputList[3].getValues().getMin()[0]-inputList[3].getValues().getMin()[0]<1e-4 else print('Problem')  
        var_Fnor = inputList[4].getValues().getMax()[0] if  inputList[4].getValues().getMin()[0]-inputList[4].getValues().getMin()[0]<1e-4 else print('Problem')   
        return field_E+210000,  field_D+10, var_Rho+7850, var_Fpos+500,  var_Fnor+100
#Here we add the means of the processes

In [4]:
# Example function imports
import PureBeamExample as pbe
import SobolIndicesAlgorithmField as siaf

##### First tests with new architecture. 
our base is a list of Karhunen Loeve Results. This will be passed into a contructor called AggregatedKarhunenLoeveResults. This  class is necessary as it is used to pass between the one dimensional space of the KarhunenLoeve Coefficients to the arbitrary dimension of the field linked to it. 
This class keeps all the data about the decomposition and allows to work on lists of processes

In [5]:
AggregatedKLObj = siaf.AggregatedKarhunenLoeveResults(listOfKLRes)

In [6]:
# Now we only need a vector of centered normal distributions to mimic the entire behaviour of our 2 fields
# and 3 random variables
Vector = ot.ComposedDistribution([ot.Normal()]*AggregatedKLObj.getSizeModes())

##### Now we can use our wrapper to transform our example function, the model of the bending beam, wher 2 of it's parameters are stochastic_fields . 

In [7]:
## First beam initialization. 
PureBeam = pbe.PureBeam(grid)
PureBeam.convertSinglInputs = convertInputs #overloading the method to pass from meshes to scalars

In [8]:
# Now that we have our function, we can pass it in our wrapper! 
FUNC = siaf.KarhunenLoeveGeneralizedFunctionWrapper(AggregatedKarhunenLoeveResults=AggregatedKLObj,
                                             func=PureBeam.singleEval, func_sample=PureBeam.batchEval,
                                             outputDim=2)
FUNC.__repr__()

"OpenTURNSPythonPointToFieldFunction( ['E__0', 'E__1', 'E__2', 'E__3', 'E__4', 'E__5', 'E__6', 'D__0', 'D__1', 'D__2', 'D__3', 'D__4', 'D__5', 'D__6', 'D__7', 'Rho__0', 'Fpos__0', 'Fnorm__0'] #18 ) -> ['y0', 'y1'] #2"

### Now that we have the new representation of our function, we can begin to create our experiment. 
For this, we use our class called **KarhunenLoeveSobolIndicesExperiment**. This last class aims to be similar to the class SobolIndicesExperiment. 

In [9]:
from importlib import reload
reload(siaf)
SobolExp = siaf.KarhunenLoeveSobolIndicesExperiment(AggregatedKLObj, 100)

In [10]:
experiment = SobolExp.generate()

     [ X0           X1           X2           X3           X4           X5           X6           X7           X8           X9           X10          X11          X12          X13          X14          X15          X16          X17          ]
 0 : [  0.608202     0.858269    -0.383208     0.956583     0.656819     0.182045    -0.883295     2.04644     -1.26618     -0.822581    -0.0343896   -1.47365      0.0438062   -1.02438      1.14606     -0.28013     -0.811068    -1.30597     ]
 1 : [ -1.26617     -0.906566    -0.355239    -0.55536      1.27211      0.466031     0.0120669    0.156523    -0.725585    -1.72433     -0.564641     0.860082    -2.28754      1.34875      0.149395     0.0311988   -0.0898914   -1.2663      ]
 2 : [ -0.438266    -0.91681     -1.3053       0.0325259    1.92925     -2.404       -0.0224003    1.11464     -1.21373      0.942426    -0.634385     0.318594    -0.988605     0.165867     0.819405    -1.42375      0.504364     0.515794    ]
 3 : [  1.20548      1.16322

In [11]:
evaluation = FUNC(experiment)

AttributeError: 'ProcessSample' object has no attribute 'setField'

In [ ]:
evaluation